In [1]:
from pyspark.sql import SparkSession
import pyspark
import re

In [53]:
from pyspark.context import SparkContext
#sc = SparkSession.builder \
#.master(sys.argv[1]) \
#.appName("XXXX") \
#.config("spark.executor.uri", "hdfs://147.213.75.180/user/hadmin/pyspark.tgz") \
#.getOrCreate()
sc = SparkContext.getOrCreate()
rdd = sc.textFile("freebase-head-100000000") #"freebase-rdf-latest.gzip"
#rdd

In [54]:
rdd.take(10)

['<http://rdf.freebase.com/ns/american_football.football_player.footballdb_id>\t<http://rdf.freebase.com/ns/type.object.type>\t<http://rdf.freebase.com/ns/type.property>\t.',
 '<http://rdf.freebase.com/ns/american_football.football_player.footballdb_id>\t<http://rdf.freebase.com/ns/type.object.name>\t"footballdb ID"@en\t.',
 '<http://rdf.freebase.com/ns/american_football.football_player.footballdb_id>\t<http://rdf.freebase.com/ns/type.property.unique>\t"true"\t.',
 '<http://rdf.freebase.com/ns/american_football.football_player.footballdb_id>\t<http://rdf.freebase.com/ns/type.property.expected_type>\t<http://rdf.freebase.com/ns/type.enumeration>\t.',
 '<http://rdf.freebase.com/ns/american_football.football_player.footballdb_id>\t<http://www.w3.org/2000/01/rdf-schema#label>\t"footballdb ID"@en\t.',
 '<http://rdf.freebase.com/ns/american_football.football_player.footballdb_id>\t<http://rdf.freebase.com/ns/type.property.schema>\t<http://rdf.freebase.com/ns/american_football.football_player

In [92]:
#toto preskocit
# common_predicate_types = sc.textFile("common-predicate-types.txt").collect()
# type_predicates = sc.textFile("type-predicates.txt").collect()

In [56]:
import re
class FileParser:
    id_pattern = '\/ns\/([^>]+)>..$'
    def __init__(self):
        pass
    
    def extract_object(self, row):
        id = self.extract_id(row)
        pattern = '".*"...'
        if(re.search(pattern, row) is not None):
            return id, re.search(pattern, row).group(0)
        else:
            return id, ''
    
    def extract_id(self, row):
        to_id = row.split('\t')[0]
        id = to_id[::-1].split('/')[0]
        id = id[::-1].split('>')[0]
        return id
        
    def extract_predicate(self, row):
        to_predicate = row.split('\t')[1]
        predicate = to_predicate[::-1].split('/')[0]
        return predicate[::-1].split('>')[0]
    
    def extract_object_id(self, row):
        pattern = '\/ns\/([^>]+)>..$'
        if re.search(pattern, row) is not None:
            found_object_id = re.search(pattern, row).group(0)
            dirty_object_id = found_object_id.split('/')[2]
            return dirty_object_id.split('>')[0]
        else:
            return None

    def text_mapper(row):
        haystack_id = parser.extract_object_id(row)
        haystack_data = haystack[haystack_id]
        triplet_parts = row.split('\t')
        pattern = '\/ns\/([^>]+)'
        film_id_dirty = re.search(pattern, triplet_parts[0]).group(0)
        film_predicate_dirty = re.search(pattern, triplet_parts[1]).group(0)
        film_id_clean = film_id_dirty.split('/')[2]
        film_predicate_clean = film_predicate_dirty.split('/')[2]
        return film_id_clean, {film_predicate_clean: haystack_data}

    def merge_two_dicts(x,y):
        z = x.copy()
        z.update(y)
        return z

parser = FileParser()    
predicates_common_alias = rdd.filter(lambda x: parser.extract_predicate(x) == 'common.alias.name')\
    .map(parser.extract_object).filter(lambda x: x[1] != '').collectAsMap()
predicates_object_name = rdd.filter(lambda x: parser.extract_predicate(x) == 'type.object.name')\
    .map(parser.extract_object).filter(lambda x: x[1] != '').collectAsMap()
haystack = predicates_common_alias.copy()
haystack.update(predicates_object_name)
entity_film = rdd.filter(lambda x: 'film' in parser.extract_predicate(x))
found = entity_film.filter(lambda x: parser.extract_object_id(x) in haystack)
matched_data = found.map(text_mapper).reduceByKey(merge_two_dicts).collectAsMap()
matched_data

{'m.0111bg3n': {'film.performance.film': '"Sequoia"@en'},
 'm.04lf87q': {'film.film.genre': '"Zenés dráma"@hu',
  'film.film.directed_by': '"Suhail Tatari"@en'},
 'm.0gc3jgs': {'film.personal_film_appearance.person': '"Al Sharpton"@es'},
 'm.0gz1tks': {'film.performance.actor': '"Victory\'s Between the Buried"@en'},
 'm.0hhgwv4': {'film.performance.film': '"يافاروم نالام"@ar'},
 'm.07zqldr': {'film.performance.actor': '"Trystian Sweet"@en'},
 'm.0r8g_r4': {'film.performance.character': '"Bronson"@en'},
 'm.064p_ws': {'film.film.story_by': '"Johan Bargum"@sv',
  'film.film.film_festivals': '"Festival de Cannes 1980"@fr'},
 'm.0hp8gvg': {'film.performance.film': '"Abenteuer mit Blasius"@en'},
 'm.0n4qw1s': {'film.film.edited_by': '"Hélène Girard"@fr'},
 'm.0vtj_c2': {'film.performance.character': '"Minister"@en'},
 'm.09l48w8': {'film.performance.actor': '"Charlotte Burton"@fr'},
 'm.0g9qg9m': {'film.film_festival_event.films': '"The Aristocrat"@en'},
 'm.0gdndd_': {'film.performance.fil

In [68]:
import pickle


with open('clean_data.pickle', 'wb') as handle:
    pickle.dump(matched_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('clean_data.pickle', 'rb') as handle:
    b = pickle.load(handle)

print(matched_data == b)

True


In [66]:
import requests

resp = requests.get('http://192.168.1.10:9200/_cat/nodes?v&pretty')

resp.text

'ip         heap.percent ram.percent cpu load_1m load_5m load_15m node.role  master name\n172.17.0.3           55          53   0    0.05    0.02     0.00 cdhilmrstw *      4c2945060b59\n'

{'m.0111bg3n': {'film.performance.film': '"Sequoia"@en'}, 'm.04lf87q': {'film.film.genre': '"Zenés dráma"@hu', 'film.film.directed_by': '"Suhail Tatari"@en'}, 'm.0gc3jgs': {'film.personal_film_appearance.person': '"Al Sharpton"@es'}, 'm.0gz1tks': {'film.performance.actor': '"Victory\'s Between the Buried"@en'}, 'm.0hhgwv4': {'film.performance.film': '"يافاروم نالام"@ar'}, 'm.07zqldr': {'film.performance.actor': '"Trystian Sweet"@en'}, 'm.0r8g_r4': {'film.performance.character': '"Bronson"@en'}, 'm.064p_ws': {'film.film.story_by': '"Johan Bargum"@sv', 'film.film.film_festivals': '"Festival de Cannes 1980"@fr'}, 'm.0hp8gvg': {'film.performance.film': '"Abenteuer mit Blasius"@en'}, 'm.0n4qw1s': {'film.film.edited_by': '"Hélène Girard"@fr'}, 'm.0vtj_c2': {'film.performance.character': '"Minister"@en'}, 'm.09l48w8': {'film.performance.actor': '"Charlotte Burton"@fr'}, 'm.0g9qg9m': {'film.film_festival_event.films': '"The Aristocrat"@en'}, 'm.0gdndd_': {'film.performance.film': '"The Holy Ch